### Structural Estimation Q5

First, we once again install some required packages and load the data.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
%matplotlib inline

ms = pd.read_csv("MacroSeries.txt", header = None, names = ["c","k","w","r"])

Next we define functions for simulating moments from the model, as well as computing our error vector.

In [130]:
def momsim(a, b, rho, mu, sigma, kdata, S, T=100):
    k1 = np.mean(kdata)
    moments = np.zeros((S, 6))
    
    for i in range(S):
        #Generating z process
        zvec = np.zeros(T)
        epsvec = np.random.normal(0, sigma, T)
        zvec[0] = mu
        for k in range(1,T):
            zvec[k] = rho* zvec[k-1] + (1-rho)*mu + epsvec[k]
        #print(zvec)

        # Generating k process
        kvec = np.zeros(T)
        k1vec = np.zeros(T)
        kvec[0] = k1
        for j in range(1,T):
            kvec[j] = a*b*np.exp(zvec[j-1])*(kvec[j-1]**(a))
        k1vec[:-1] = kvec[1:]
        k1vec[T-1] = a*b*np.exp(zvec[T-1])*(kvec[T-1]**(a))


        # Backing out w
        wvec = (1-a)*np.exp(zvec)*(kvec **(a))

        # Backing out r
        rvec = a*np.exp(zvec)*(kvec **(a-1))

        # Backing out c
        cvec = wvec + rvec*kvec - k1vec
    
        moments[i,0] = np.mean(cvec)
        moments[i,1] = np.mean(kvec)
        moments[i,2] = np.var(cvec)
        moments[i,3] = np.var(kvec)
        moments[i,4] = np.corrcoef(kvec, cvec)[0,1]
        moments[i,5] = np.corrcoef(kvec[:-1], kvec[1:])[0,1]
        
    val = np.zeros(6)
    for l in range(6):
        val[l] = np.mean(moments[:,l])
    return val

def datamoments(kvec, cvec):
    datamoments = np.zeros(6)
    datamoments[0] = np.mean(cvec)
    datamoments[1] = np.mean(kvec)
    datamoments[2] = np.var(cvec)
    datamoments[3] = np.var(kvec)
    datamoments[4] = np.corrcoef(kvec, cvec)[0,1]
    datamoments[5] = np.corrcoef(kvec[:-1], kvec[1:])[0,1]
    return datamoments

def errvec(modelmom, datamom):
    errvec = abs(datamom - modelmom)/datamom
    return errvec

def crit(params, *args):
    a, b, rho, mu, sigma = params
    kdata, cdata, W = args
    modelmoments = momsim(a, b, rho, mu, sigma, kdata, S = 1000)
    datamom = datamoments(kdata, cdata)
    err = errvec(modelmoments, datamom)
    crit = err.dot(W.dot(err)) 
    return crit
    
    

In [136]:
# Lets attempt to compute the SMM estimator

params = (0.45, 0.97, 0.7, 10, 0.5)
args = (ms['k'], ms['c'], np.eye(6))

results = opt.minimize(crit, params, args, method='L-BFGS-B', tol =1e-8,
                       bounds=((0.01,0.99), (0.01,0.99),(-0.99,0.99), (-0.5, 1),(0.001,1)))

In [135]:
results

 final_simplex: (array([[ 0.37083526,  1.03068975,  0.73162974,  9.32189378,  0.53361793],
       [ 0.37083526,  1.03068975,  0.73162974,  9.32189378,  0.53361793],
       [ 0.37083526,  1.03068975,  0.73162974,  9.32189378,  0.53361793],
       [ 0.37083526,  1.03068975,  0.73162974,  9.32189378,  0.53361793],
       [ 0.37083526,  1.03068975,  0.73162974,  9.32189378,  0.53361793],
       [ 0.37083526,  1.03068975,  0.73162974,  9.32189378,  0.53361793]]), array([ 1.52143489,  1.54255552,  1.54773264,  1.5487955 ,  1.6050616 ,
        1.74503278]))
           fun: 1.5214348895633127
       message: 'Maximum number of function evaluations has been exceeded.'
          nfev: 1003
           nit: 376
        status: 1
       success: False
             x: array([ 0.37083526,  1.03068975,  0.73162974,  9.32189378,  0.53361793])

Right now it still seems that the function does not converge when put through the iterative solver. Further work on this is pending...